In [1]:
import os

from series_extraction.excel_loader import ExcelLoader
from series_extraction.excel_cleaner import ExcelCleaner
from series_extraction.table_extractor import TableExtractor
from series_extraction.series_extractor import SeriesExtractor
from series_extraction.excel_compatibility_checker import ExcelCompatibilityChecker
from series_extraction.series_iterator import SeriesIterator
from series_extraction.series_mapper import SeriesMapper
from series_extraction.excel_validator import ExcelValidator

from ast_transformation.series_formula_generator_old import SeriesFormulaGenerator
from ast_transformation.formula_generator import FormulaGenerator

from ast_building.formula_parser import FormulaParser

from pipeline_building.series_dependencies_builder import SeriesDependenciesBuilder
from pipeline_building.dag_sorter import DAGSorter
from pipeline_building.pipeline_builder import PipelineBuilder

from excel_builder import ExcelBuilder

from excel_checker import ExcelChecker

from excel_data_extractor import ExcelDataExtractor


In [2]:
data_directory = "/Users/chrislittle/GitHub/speedsheet/excel-2-python/data"

project_name = 'vehicle_data'

excel_raw_file_path = os.path.join(data_directory, "excel_files_raw", f"{project_name}_raw.xlsx")
excel_reduced_filepath = os.path.join(data_directory, "excel_files_reduced", f"{project_name}_reduced.xlsx")
excel_reduced_clean_filepath = os.path.join(data_directory, "excel_files_reduced_clean", f"{project_name}_reduced_clean.xlsx")
excel_raw_clean_filepath = os.path.join(data_directory, "excel_files_raw_clean", f"{project_name}_raw_clean.xlsx")
excel_reduced_clean_series_filepath = os.path.join(data_directory, "excel_files_reduced_clean_series", f"{project_name}_reduced_clean_series.xlsx")
excel_reduced_clean_series_python_filepath = os.path.join(data_directory, "excel_files_reduced_clean_series_python", f"{project_name}_reduced_clean_series_python.xlsx")
excel_raw_clean_series_python_filepath = os.path.join(data_directory, "excel_files_raw_clean_series_python", f"{project_name}_raw_clean_series_python.xlsx")

In [3]:
excel_raw= ExcelLoader.load_file(excel_raw_file_path)
excel_reduced = ExcelLoader.load_file(excel_reduced_filepath)
is_valid = ExcelValidator.validate_excel(excel_reduced)
if not is_valid:
    raise Exception("Excel file is not valid")

excel_reduced_clean = ExcelCleaner.clean_excel(excel_reduced)
excel_raw_clean = ExcelCleaner.clean_excel(excel_raw)
ExcelBuilder.create_excel_from_workbook(excel_reduced_clean.workbook_with_formulas, excel_reduced_clean_filepath)
ExcelBuilder.create_excel_from_workbook(excel_raw_clean.workbook_with_formulas,  excel_raw_clean_filepath)

extracted_tables, workbook_data = TableExtractor.extract_tables(excel_reduced_clean)
is_compatible = ExcelCompatibilityChecker.check_file(excel_raw, excel_reduced, extracted_tables)
if not is_compatible:
    raise Exception("Excel file is not compatible")

series_dict = SeriesExtractor.extract_series(extracted_tables=extracted_tables, workbook_data=workbook_data)
series_mapping = SeriesMapper.map_series(series_dict)
series_iterator = SeriesIterator.iterate_series(series_dict)

series_list = [series for series in series_iterator]

series_list_with_formulas = [
    series for series in series_list if series.formulas != [None, None]
]
series_list_with_values = [
    series for series in series_list if series.formulas == [None, None]
]

generic_formula_dictionary = {}

for series in series_list_with_formulas:
    formula_1, formula_2 = SeriesFormulaGenerator.adjust_formulas(
        series.formulas
    )
    if formula_1 is not None and formula_2 is not None:

        print(f"formula_1: {formula_1}")
        print(f"formula_2: {formula_2}")

        formula_1_ast = FormulaParser.parse_formula(formula_1)
        formula_2_ast = FormulaParser.parse_formula(formula_2)

        # generic_formula_ast = FormulaGenerator.generate(formula_1_ast_series, formula_2_ast_series)

        # generic_formula_dictionary[series.series_id] = generic_formula_ast



In [18]:
formula_1

'=IF(SUM(Y2:AB2)=4,1,0)'

In [16]:
formula_1_ast

([IF <Function>,
  ( <Parenthesis>,
  SUM <Function>,
  ( <Parenthesis>,
  Y2:AB2 <Range>,
  ) <Parenthesis>,
  = <Operator>,
  4 <Number>,
  , <Operator>,
  1 <Number>,
  , <Operator>,
  0 <Number>,
  ) <Parenthesis>],
 <formulas.builder.AstBuilder at 0x152991e50>)

In [4]:
series_values_dict = {str(series.series_id): series.values for series in series_list}

In [5]:
series_values_dict_raw = ExcelDataExtractor.extract_series_data_from_excel(excel_raw.workbook_with_values, series_list)

In [6]:
series_dependencies = SeriesDependenciesBuilder.build_dependencies(generic_formula_dictionary)

In [7]:
sorted_dag = DAGSorter.sort_dag(series_dependencies)

In [8]:
series_list_updated_raw = PipelineBuilder.create_series_list(sorted_dag, generic_formula_dictionary, series_dict, series_values_dict_raw, series_list_with_values)

In [9]:
series_list_updated_raw

[Series(series_id=SeriesId(sheet_name='VehicleData', series_header='vrm', series_header_cell_row=1, series_header_cell_column=1), worksheet=Worksheet(sheet_name='VehicleData', workbook_file_path=None, worksheet=None), series_header='vrm', formulas=[None, None], values=['YY70KTT', 'YY15UUW', 'YY11DCF', 'YX63UFY', 'YW67DTF', 'YT65XVL', 'YT12LML', 'YS67HKZ', 'YS64XPB', 'YR66UYL', 'YR64NVJ', 'YR64HWT', 'YR63VSG', 'YR61KVF', 'YR59NLO', 'YR13VOH', 'YR10FPZ', 'YP60UJA', 'YP22RXB', 'YP22RXA', 'YP15TOV', 'YP14XBT', 'YP12KCX', 'YO18AKX', 'YO15LVP', 'YM15UFH', 'YM14KOH', 'YL14EPU', 'YK67KPZ', 'YK65UAM', 'YK60MDY', 'YK13ETU', 'YK13AOR', 'YK11WNB', 'YK11VRV', 'YJ64XHZ', 'YH72VNT', 'YH65FAA', 'YH61WUP', 'YH18LGN', 'YH17VWM', 'YH15AHJ', 'YG65OGT', 'YG64HSU', 'YG62YHU', 'YG61XHJ', 'YG61XCK', 'YG13CFY', 'YF68OBA', 'YF17YVC', 'YF14ZBG', 'YF11YJA', 'YE68JDA', 'YE64BSX', 'YE16WEW', 'YE15CXT', 'YE14LUL', 'YD66YUT', 'YD63LZM', 'YD21PPE', 'YD13HYZ', 'YD12TLZ', 'YC72BUV', 'YC64MKK', 'YC63UZW', 'YC22WBP', 'YC1

In [10]:
series_list_updated_reduced = PipelineBuilder.create_series_list(sorted_dag, generic_formula_dictionary, series_dict, series_values_dict, series_list_with_values)

In [11]:
series_list_updated_reduced

[Series(series_id=SeriesId(sheet_name='VehicleData', series_header='vrm', series_header_cell_row=1, series_header_cell_column=1), worksheet=Worksheet(sheet_name='VehicleData', workbook_file_path=None, worksheet=None), series_header='vrm', formulas=[None, None], values=['YY70KTT', 'YY15UUW'], series_starting_cell=Cell(column=1, row=2, coordinate='A2', sheet_name=None, value=None, value_type=None, formula=None), series_length=2, series_data_type=<SeriesDataType.STR: 'str'>),
 Series(series_id=SeriesId(sheet_name='VehicleData', series_header='make', series_header_cell_row=1, series_header_cell_column=2), worksheet=Worksheet(sheet_name='VehicleData', workbook_file_path=None, worksheet=None), series_header='make', formulas=[None, None], values=['Kia', 'Vauxhall'], series_starting_cell=Cell(column=2, row=2, coordinate='B2', sheet_name=None, value=None, value_type=None, formula=None), series_length=2, series_data_type=<SeriesDataType.STR: 'str'>),
 Series(series_id=SeriesId(sheet_name='Vehicl

In [12]:
ExcelBuilder.create_excel_from_series(series_list_updated_reduced, excel_reduced_clean_series_python_filepath, values_only=True)

In [13]:
ExcelBuilder.create_excel_from_series(series_list_updated_raw, excel_raw_clean_series_python_filepath, values_only=True)

In [14]:
ExcelChecker.excels_are_equivalent(excel_reduced_clean_series_python_filepath, excel_reduced_clean_filepath)

True

In [15]:
ExcelChecker.excels_are_equivalent(excel_raw_clean_series_python_filepath, excel_raw_clean_filepath)

True